In [4]:
from pydantic import BaseModel, Field
import json
from enum import Enum
from typing import Dict, List, Optional, Any, Union

# Helper function to convert Pydantic schema to Gemini schema format
def convert_to_gemini_schema(pydantic_schema: Dict[str, Any]) -> Dict[str, Any]:
    gemini_schema = {}
    
    # Map type
    if "type" in pydantic_schema:
        gemini_schema["type"] = pydantic_schema["type"]
    
    # Map format if available
    if "format" in pydantic_schema:
        gemini_schema["format"] = pydantic_schema["format"]
    
    # Map description
    if "description" in pydantic_schema:
        gemini_schema["description"] = pydantic_schema["description"]
    
    # Map nullable (Pydantic uses 'nullable' in JSON Schema)
    if any(ref == "null" for ref in pydantic_schema.get("anyOf", [])):
        gemini_schema["nullable"] = True
    
    # Map enum values
    if "enum" in pydantic_schema:
        gemini_schema["enum"] = pydantic_schema["enum"]
    
    # Map array constraints
    if "maxItems" in pydantic_schema:
        gemini_schema["maxItems"] = str(pydantic_schema["maxItems"])
    if "minItems" in pydantic_schema:
        gemini_schema["minItems"] = str(pydantic_schema["minItems"])
    
    # Map properties
    if "properties" in pydantic_schema:
        gemini_schema["properties"] = {}
        for prop_name, prop_schema in pydantic_schema["properties"].items():
            gemini_schema["properties"][prop_name] = convert_to_gemini_schema(prop_schema)
    
    # Map required fields
    if "required" in pydantic_schema:
        gemini_schema["required"] = pydantic_schema["required"]
    
    # Map items for arrays
    if "items" in pydantic_schema:
        gemini_schema["items"] = convert_to_gemini_schema(pydantic_schema["items"])
    
    # Add propertyOrdering if needed
    if "properties" in pydantic_schema:
        gemini_schema["propertyOrdering"] = list(pydantic_schema["properties"].keys())
    
    return gemini_schema